In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Default crop dimensions (3:4 aspect ratio)
CROP_WIDTH = 300
CROP_HEIGHT = 400

# Globals
img = None
tk_img = None
image_path = None
updating = False

crop_center = None
drag_start_x = None
min_crop_width = 60
max_crop_width = 600
rect_id = None

def select_image():
    global img, tk_img
    file_path = filedialog.askopenfilename(
        title="Select an image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    if file_path:
        load_image(file_path)

def load_image(path):
    global img, tk_img, canvas, image_path, rect_id, crop_center
    image_path = path
    img = Image.open(path).convert("RGBA")  # Handle transparency

    max_width, max_height = 900, 1200
    if img.width > max_width or img.height > max_height:
        img.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)

    tk_img = ImageTk.PhotoImage(img)
    canvas.config(width=tk_img.width(), height=tk_img.height())
    canvas.delete("all")
    canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)

    crop_center = None
    if rect_id is not None:
        canvas.delete(rect_id)
        rect_id = None

    canvas.bind("<Button-1>", on_click)
    canvas.bind("<B1-Motion>", on_drag)
    canvas.bind("<ButtonRelease-1>", on_release)

def on_click(event):
    global crop_center, drag_start_x
    crop_center = (event.x, event.y)
    drag_start_x = event.x
    update_crop_rect()

def on_drag(event):
    global drag_start_x
    if crop_center is None:
        return

    dx = event.x - drag_start_x
    drag_start_x = event.x

    try:
        current_width = int(width_entry.get())
    except ValueError:
        current_width = CROP_WIDTH

    new_width = max(min_crop_width, min(max_crop_width, current_width + dx))
    new_height = int(new_width * 4 / 3)

    global updating
    updating = True
    width_var.set(str(new_width))
    height_var.set(str(new_height))
    updating = False

    update_crop_rect()

def on_release(event):
    pass

def update_crop_rect():
    global rect_id, crop_center
    if crop_center is None:
        return

    try:
        crop_width = int(width_entry.get())
        crop_height = int(height_entry.get())
    except ValueError:
        return

    left = crop_center[0] - crop_width // 2
    top = crop_center[1] - crop_height // 2
    right = crop_center[0] + crop_width // 2
    bottom = crop_center[1] + crop_height // 2

    left = max(0, left)
    top = max(0, top)
    right = min(canvas.winfo_width(), right)
    bottom = min(canvas.winfo_height(), bottom)

    if rect_id is not None:
        canvas.delete(rect_id)

    rect_id = canvas.create_rectangle(
        left, top, right, bottom,
        outline="red", width=2
    )

def crop_centered_3_4_img(image, center_x, center_y):
    width, height = image.size

    try:
        crop_width = int(width_entry.get())
        crop_height = int(height_entry.get())
    except ValueError:
        print("Invalid crop dimensions.")
        return

    left = max(0, center_x - crop_width // 2)
    upper = max(0, center_y - crop_height // 2)
    right = min(width, center_x + crop_width // 2)
    lower = min(height, center_y + crop_height // 2)

    if right - left < crop_width:
        if left == 0:
            right = min(width, crop_width)
        elif right == width:
            left = max(0, width - crop_width)

    if lower - upper < crop_height:
        if upper == 0:
            lower = min(height, crop_height)
        elif lower == height:
            upper = max(0, height - crop_height)

    cropped = image.crop((left, upper, right, lower))

    # Handle image mode for saving
    if image_path.lower().endswith(".png"):
        save_path = "cropped_result.png"
        cropped.save(save_path)
    else:
        save_path = "cropped_result.jpg"
        cropped = cropped.convert("RGB")
        cropped.save(save_path)

    print(f"Cropped image saved as {save_path}")

    preview = tk.Toplevel()
    preview.title("Cropped Preview")
    cropped_tk = ImageTk.PhotoImage(cropped)
    label = tk.Label(preview, image=cropped_tk)
    label.image = cropped_tk
    label.pack()

def update_height(event=None):
    global updating
    if updating:
        return
    try:
        updating = True
        w = int(width_entry.get())
        h = int(w * 4 / 3)
        height_var.set(str(h))
    except ValueError:
        pass
    finally:
        updating = False
    update_crop_rect()

def update_width(event=None):
    global updating
    if updating:
        return
    try:
        updating = True
        h = int(height_entry.get())
        w = int(h * 3 / 4)
        width_var.set(str(w))
    except ValueError:
        pass
    finally:
        updating = False
    update_crop_rect()

def on_double_click(event):
    if crop_center is not None and img is not None:
        crop_centered_3_4_img(img, crop_center[0], crop_center[1])

# GUI setup
root = tk.Tk()
root.title("3:4 Crop Tool with Drag Resize")

canvas = tk.Canvas(root)
canvas.pack()

frame = tk.Frame(root)
frame.pack(pady=5)

tk.Button(frame, text="Select Image", command=select_image).grid(row=0, column=0, padx=5)

width_var = tk.StringVar(value=str(CROP_WIDTH))
height_var = tk.StringVar(value=str(CROP_HEIGHT))

tk.Label(frame, text="Width:").grid(row=0, column=1)
width_entry = tk.Entry(frame, textvariable=width_var, width=6)
width_entry.grid(row=0, column=2)
width_entry.bind("<KeyRelease>", update_height)

tk.Label(frame, text="Height:").grid(row=0, column=3)
height_entry = tk.Entry(frame, textvariable=height_var, width=6)
height_entry.grid(row=0, column=4)
height_entry.bind("<KeyRelease>", update_width)

canvas.bind("<Double-Button-1>", on_double_click)

root.mainloop()